In [ ]:
import numpy as np
import pandas as pd
import plotly as plt
import scipy
import math
from datetime import datetime

In [ ]:
df_matches = pd.read_csv("../data/cleaned/tennis_matches_cleaned.csv", parse_dates=['tourney_date'])
df_male_players = pd.read_csv("../data/cleaned/male_players_cleaned.csv")
df_female_players = pd.read_csv("../data/cleaned/female_players_cleaned.csv")

# Data integration

Create a dataframe with all the male matches and another one for the female matches

In [ ]:
df_male_players_tmp = pd.DataFrame()
df_male_players_tmp['player_name'] = df_male_players['name'] + ' ' + df_male_players['surname']
df_male_winners = df_male_players_tmp.join(df_matches.set_index('winner_name'), on='player_name', how='inner')
df_male_losers = df_male_players_tmp.join(df_matches.set_index('loser_name'), on='player_name', how='inner')
df_male_matches = pd.concat([df_male_winners, df_male_losers])

In [ ]:
df_male_matches

,player_name,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,loser_rank,tourney_spectators,tourney_revenue,winner_name
643,Alexander Zverev,352,2019-580,Australian Open,Hard,128,G,2019-01-14,200,100644,...,37.0,12.0,13.0,6.0,12.0,4,155,15583.0,2781000.18,NaN
643,Alexander Zverev,363,2019-580,Australian Open,Hard,128,G,2019-01-14,116,100644,...,34.0,8.0,13.0,2.0,8.0,4,67,15583.0,2781000.18,NaN
643,Alexander Zverev,364,2019-580,Australian Open,Hard,128,G,2019-01-14,172,100644,...,86.0,29.0,26.0,12.0,17.0,4,36,15583.0,2781000.18,NaN
643,Alexander Zverev,1145,2019-M004,Acapulco,Hard,32,A,2019-02-25,298,100644,...,29.0,8.0,10.0,2.0,4.0,3,64,3398.0,655207.08,NaN
643,Alexander Zverev,1149,2019-M004,Acapulco,Hard,32,A,2019-02-25,294,100644,...,28.0,11.0,10.0,0.0,2.0,3,26,3398.0,655207.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54476,Michele Vianello,94508,2021-9357,Forli CH,Clay,32,C,2021-06-14,243,104563,...,21.0,9.0,9.0,2.0,6.0,433,-1,3031.0,566031.89,Luca Vanni
54476,Michele Vianello,95338,2021-2120,Cordenons CH,Clay,32,C,2021-08-02,250,207608,...,28.0,5.0,10.0,2.0,7.0,291,-1,2876.0,543907.53,Timofey Skatov
54485,Ben Shelton,95003,2021-7316,Cary CH,Hard,32,C,2021-07-19,254,206499,...,24.0,12.0,9.0,3.0,5.0,413,1835,3074.0,583894.56,Aleksandar Kovacevic
54504,William Jansen,64393,2021-540,Wimbledon,Grass,128,G,2021-06-28,138,207760,...,28.0,17.0,13.0,8.0,16.0,454,-1,14566.0,5002793.97,Anton Matusevich


df_male_matches.describe()

In [ ]:
df_female_players_tmp = pd.DataFrame()
df_female_players_tmp['player_name'] = df_female_players['name'] + ' ' + df_female_players['surname']
df_female_winners = df_female_players_tmp.join(df_matches.set_index('winner_name'), on='player_name', how='inner')
df_female_losers = df_female_players_tmp.join(df_matches.set_index('loser_name'), on='player_name', how='inner')
df_female_matches = pd.concat([df_female_winners, df_female_losers])

In [ ]:
df_male_matches_reduced = df_male_losers[['player_name', 'winner_id', 'winner_ht', 'winner_age', 'tourney_date']]
df_male_matches_reduced[df_male_matches_reduced['player_name'] == 'Aaron Addison']

,player_name,winner_id,winner_ht,winner_age,tourney_date
31893,Aaron Addison,117357,-1,19,2016-10-24
31893,Aaron Addison,117360,-1,18,2016-02-01
31893,Aaron Addison,102864,193,18,2016-01-11
31893,Aaron Addison,105429,-1,26,2016-10-31
31893,Aaron Addison,111806,-1,23,2020-01-13
31893,Aaron Addison,105349,-1,30,2019-10-21
31893,Aaron Addison,106172,-1,25,2019-01-07
31893,Aaron Addison,200614,-1,21,2019-10-28
31893,Aaron Addison,200640,-1,19,2018-10-22


- name
- hand
- sex
- nationality
- win percentage
- tot match
- number of won tournment

- year with more matches
- year with less matches
- year with more wins
- year with less wins

Reset the index of both the new data frames

In [ ]:
df_male_matches.reset_index(drop=True, inplace=True)
df_female_matches.reset_index(drop=True, inplace=True)

In [ ]:
df_player = pd.DataFrame()

list_of_male_names = df_male_matches['player_name'].unique()
#list_of_male_names = df_male_players_tmp['player_name'].unique()
print(f"Lunghezza lista: {len(list_of_male_names)}")
df_player['Name'] = list_of_male_names
df_player['Sex'] = 'M'
i = 0

for name in list_of_male_names:
    print(i)
    i+=1
    df_player_matches = df_matches[(df_matches['winner_name'] == name) | (df_matches['loser_name'] == name)]

    #player's hand
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_hand'] != 'U')]['winner_hand']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_hand'] != 'U')]['loser_hand']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'Hand'] = 'U'

    #player's nationality
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_ioc'] != None)]['winner_ioc']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_ioc'] != 'U')]['loser_ioc']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'IOC'] = 'Undefined'
    
    df_player.loc[df_player['Name'] == name, 'Won_Tournaments'] = df_player_matches[(df_player_matches['round'] == 'F') & (df_player_matches['winner_name'] == name)].shape[0]

    if df_player_matches.shape[0] != 0:
        df_player.loc[df_player['Name'] == name, 'Number_of_Matches'] = df_player_matches.shape[0]
        df_player.loc[df_player['Name'] == name, 'Win_Percentage'] = df_player_matches[df_player_matches['winner_name'] == name].shape[0]/df_player_matches.shape[0]
    
    match_per_year = df_male_matches[df_male_matches['player_name']==name]['tourney_date'].groupby(df_male_matches['tourney_date'].dt.year).count()
    if len(match_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_matches'] = match_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_matches'] = match_per_year.idxmin()

    wins_per_year = df_male_matches[df_male_matches['winner_name']==name]['tourney_date'].groupby(df_male_matches['tourney_date'].dt.year).count()
    if len(wins_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_wins'] = wins_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_wins'] = wins_per_year.idxmin()




2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218


In [ ]:
df_player.to_csv("../data/integrations/matches_male_players.csv", index=False)

In [ ]:
df_player = pd.DataFrame()

list_of_female_names = df_female_matches['player_name'].unique()
print(f"Lunghezza lista: {len(list_of_female_names)}")
df_player['Name'] = list_of_female_names
df_player['Sex'] = 'F'
i = 0

for name in list_of_female_names:
    
    df_player_matches = df_matches[(df_matches['winner_name'] == name) | (df_matches['loser_name'] == name)]

    #player's hand
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_hand'] != 'U')]['winner_hand']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_hand'] != 'U')]['loser_hand']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'Hand'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'Hand'] = 'U'

    #player's nationality
    df_h = [x for x in df_player_matches[(df_player_matches['winner_name'] == name) &  (df_player_matches['winner_ioc'] != None)]['winner_ioc']]
    if len(df_h) > 0:
        df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
    else:
        df_h = [x for x in df_player_matches[(df_player_matches['loser_name'] == name) &  (df_player_matches['loser_ioc'] != 'U')]['loser_ioc']]
        if len(df_h) > 0:
            df_player.loc[df_player['Name'] == name, 'IOC'] = df_h[0]
        else:
            df_player.loc[df_player['Name'] == name, 'IOC'] = 'Undefined'
    
    df_player.loc[df_player['Name'] == name, 'Won_Tournaments'] = df_player_matches[(df_player_matches['round'] == 'F') & (df_player_matches['winner_name'] == name)].shape[0]

    match_per_year = df_male_matches[df_male_matches['player_name']==name]['tourney_date'].groupby(df_male_matches['tourney_date'].dt.year).count()
    if len(match_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_matches'] = match_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_matches'] = match_per_year.idxmin()

    wins_per_year = df_male_matches[df_male_matches['winner_name']==name]['tourney_date'].groupby(df_male_matches['tourney_date'].dt.year).count()
    if len(wins_per_year) > 0:
        df_player.loc[df_player['Name'] == name, 'best_year_wins'] = wins_per_year.idxmax()
        df_player.loc[df_player['Name'] == name, 'worst_year_wins'] = wins_per_year.idxmin()

Lunghezza lista: 7062


In [ ]:
df_player.to_csv("../data/integrations/matches_female_players.csv", index=False)

## Start from here

In [ ]:
male_stats = pd.read_csv("../data/integrations/matches_male_players.csv")
female_stats = pd.read_csv("../data/integrations/matches_female_players.csv")

Idee:

- Dividere le statistiche dei player in base al tipo di surface dove giocano, come fatto nel sito https://www.atptour.com/en/players/novak-djokovic/d643/player-stats
- Fare un'analisi separata maschi/femmine e poi confrontarla con le performance che ci sono nei tornei misti (se ne troviamo)



### Dataframe TOP Player

In [ ]:
TOP = 50

In [ ]:
#Input:
# - df, the dataframe you want to work on
# - attr, the attribute you want to sum row by row
# - group_by, the attribute used to group the dataframe before summing (Here we sum rows with the same "group_by" value)
# - top_list, this other dataframe is left joined with df to restrict its result only to the top_list
# - top_list_attr, it's needed in the join function to know how's called the attribute

def top_players_feature(df, attr, group_by, top_list, top_list_attr):
    #count 
    tmp = pd.DataFrame(df[df[attr]>=0].groupby(group_by, as_index=False)[attr].sum())
    #intersection with top_list
    tmp = top_list.join(tmp.set_index(group_by), on=top_list_attr, how='left')
    
    return tmp

#Function to generate the sum of a column, with a condition, grouped by the attribute given in input

def sum_by_surface(df, attr, group_by, top_list, top_list_attr):
    #count 
    tmp = pd.DataFrame(df[df[attr]>=0].groupby([group_by, 'surface'], as_index=False)[attr].sum())
    #intersection with top_list
    tmp = top_list.join(tmp.set_index(group_by), on=top_list_attr, how='left')
    
    return [tmp[tmp['surface'] == 'Clay'], tmp[tmp['surface'] == 'Grass'], tmp[tmp['surface'] == 'Hard'], tmp[tmp['surface'] == 'Carpet']]

In [ ]:
# take the top players according to number of won tournament
df_top_male_players = pd.DataFrame(male_stats.sort_values(by='Won_Tournaments', ascending=False).head(TOP)['Name'])

#iterating for every player
#for name,index in df_top_male_players.iterrows():
#df_matches.groupby('winner_name').count()['w_ace'] 

df_top_male_player_matches_win = df_matches.join(df_top_male_players.set_index('Name'), on='winner_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_male_player_matches_los = df_matches.join(df_top_male_players.set_index('Name'), on='loser_name', how='inner') #Ci sono partite in cui i winner sono loser e non vengono considerate
df_top_male_player_matches  = pd.concat([df_top_male_player_matches_win, df_top_male_player_matches_los])
#df_top_male_players['w_ace'] = sum_by_condition(df_top_male_player_matches, 'w_ace', 'winner_name', df_top_male_players, 'Name')['w_ace']
# Ciclo for su lista di attributi che chiama sum_by_condition

attrs = ['w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced']

#sum_by_condition(df_top_male_player_matches, 'w_svpt', 'winner_name', df_top_male_players, 'Name')

#sum_by_surface(df_top_male_player_matches, 'w_svpt', 'winner_name', df_top_male_players, 'Name', 'Clay')

for attr in attrs:
    #df_top_male_players[attr] = top_players_feature(df_top_male_player_matches, attr, 'winner_name', df_top_male_players, 'Name')[attr]
    result = sum_by_surface(df_top_male_player_matches, attr, 'winner_name', df_top_male_players, 'Name')
    df_top_male_players[attr + '_clay'] = result[0][attr]
    df_top_male_players[attr + '_clay'].fillna(0, inplace=True)
    df_top_male_players[attr + '_grass'] = result[1][attr]
    df_top_male_players[attr + '_grass'].fillna(0, inplace=True)
    df_top_male_players[attr + '_hard'] = result[2][attr]
    df_top_male_players[attr + '_hard'].fillna(0, inplace=True)
    df_top_male_players[attr + '_carpet'] = result[2][attr]
    df_top_male_players[attr + '_carpet'].fillna(0, inplace=True)

df_top_male_players

,Name,w_ace_clay,w_ace_grass,w_ace_hard,w_ace_carpet,w_df_clay,w_df_grass,w_df_hard,w_df_carpet,w_svpt_clay,...,w_SvGms_hard,w_SvGms_carpet,w_bpSaved_clay,w_bpSaved_grass,w_bpSaved_hard,w_bpSaved_carpet,w_bpFaced_clay,w_bpFaced_grass,w_bpFaced_hard,w_bpFaced_carpet
192,Novak Djokovic,312.0,107.0,460.0,460.0,244.0,147.0,569.0,569.0,9237.0,...,2583.0,2583.0,445.0,152.0,588.0,588.0,654.0,208.0,859.0,859.0
154,Rafael Nadal,330.0,35.0,451.0,451.0,269.0,32.0,336.0,336.0,10917.0,...,2139.0,2139.0,537.0,47.0,450.0,450.0,760.0,64.0,647.0,647.0
0,Alexander Zverev,245.0,11.0,276.0,276.0,389.0,88.0,781.0,781.0,8221.0,...,2535.0,2535.0,310.0,75.0,642.0,642.0,508.0,102.0,915.0,915.0
35,Roger Federer,52.0,107.0,280.0,280.0,27.0,64.0,278.0,278.0,1563.0,...,1943.0,1943.0,57.0,150.0,375.0,375.0,81.0,196.0,529.0,529.0
766,Daniil Medvedev,89.0,44.0,416.0,416.0,100.0,98.0,790.0,790.0,2496.0,...,2947.0,2947.0,118.0,84.0,710.0,710.0,180.0,123.0,1030.0,1030.0
679,Dominic Thiem,471.0,15.0,293.0,293.0,412.0,63.0,433.0,433.0,12133.0,...,2241.0,2241.0,517.0,47.0,542.0,542.0,765.0,60.0,805.0,805.0
188,Andy Murray,72.0,40.0,249.0,249.0,84.0,38.0,318.0,318.0,2609.0,...,1394.0,1394.0,126.0,88.0,343.0,343.0,181.0,128.0,522.0,522.0
1051,Andrey Rublev,136.0,34.0,389.0,389.0,135.0,32.0,560.0,560.0,5121.0,...,2411.0,2411.0,261.0,42.0,629.0,629.0,405.0,68.0,901.0,901.0
1366,Ugo Humbert,25.0,10.0,201.0,201.0,24.0,75.0,310.0,310.0,712.0,...,1340.0,1340.0,25.0,88.0,362.0,362.0,48.0,118.0,480.0,480.0
546,James Duckworth,20.0,19.0,138.0,138.0,27.0,26.0,250.0,250.0,713.0,...,1330.0,1330.0,49.0,63.0,314.0,314.0,62.0,76.0,425.0,425.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=de87b132-0371-4d99-94f9-c61923e2507d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>